# Analysis, Visualization and Extensive Imputation of CO2 Emissions in Food Products

<a id="section-one"></a>
# 1. Introduction

**Objectives:** The objectives of this analysis are two-fold. First, to provide an investigation of the dataset within the archetype of a scientist and researcher attempting to compress information to be used for colleagues at various positions (e.g. engineers might require more operational insight, while management will require an overview). Second, to provide a walkthrough for all fellow coders and statisticians through descriptive actions.

**Descriptive actions:** I tried to describe the reasoning behind (almost) every line of code during the examination of this dataset - especially during the feature engineering, imputation, and analysis sections. This extra work serves two functions: first to help new learners understand all featured techniques and second, to allow more experienced statisticians and coders to correct my code or way of approaching the problem - even through there are some serious limitations about this dataset.

**Limitations:** It is also important to note that since we have <50 entries there is a very high probability that many models are inadvisable to use since we would not be able to obtain a sufficient alpha level. This includes many estimator models we utilize with sklearn but also with any correlation, regression and other analysis performed. More on this later on. Finally, I tried to adhere to the Zen of Python rules, however, some have been deliberately broken in order to showcase certain analyses.

<a id="section-two"></a>
# 2. Importing Libraries and Datasets

In [ ]:
import math as math
import pandas as pd
import numpy as np
import seaborn as sns
import squarify as sqf
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge

In [ ]:
df_food = pd.read_csv("../input/environment-impact-of-food-production/Food_Production.csv")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

<a id="section-three"></a>
# 3. Overview

First, let us start in the standard way and have a quick overview of our dataset prior to feature engineering and subsequent analysis.

In [ ]:
df_food.columns

In [ ]:
df_food.head(3)

# A few first-sight findings: 
# We notice long-winded colunm names, naming inconsistencies, overwhelming usage of numeric columns, descriptive statistics will most likely revolve around the food products and certain key features/columns

In [ ]:
df_food.describe(include='all')

# Negative values can be spotted under the [Land Use Change] column (notice the minumum values)
# Incosistencies detected in the 75% percentile range for certain columns (e.g. [Animal Feed] and [Transport])
# Radical differences in min/max in latter columns (e.g. [Scarcity Water])

In [ ]:
df_food.info()

# Mostly numeric variables. Several missing values which already point towards a Missing Not At Random (MNAR) archetype.
# We shall deal with them in the Feature Engineering Section.

<a id="section-four"></a>
# 4. Feature Engineering

We have a few issues to address before we can proceed, we need to:
- First, rename our columns
- Second, scout for any negative values
- Third, deal with any missing values
- Fourth, assuming that we have taken the task to assist science, engineering, or business operations, we might need to make certain data transformations in order to assist our colleagues who will surely have varying needs (e.g. generalized vs specific, product-focused vs process-focused, etc.).

<a id="section-four"></a>
## 4.1 Renaming Columns

In [ ]:
# The column names in the dataset are quite inconsistent (some use underscores, others spaces, there are letters missing, etc.) and some columns have long names that need to be revisited.

# Creating short dataframes containing a keyword from the various columns
df_eutro = df_food.loc[:, ["Eutro" in i for i in df_food.columns]]
df_freshwater = df_food.loc[:, ["Freshwater" in i for i in df_food.columns]]
df_gas = df_food.loc[:, ["gas" in i for i in df_food.columns]]
df_land = df_food.loc[:, ["Land" in i for i in df_food.columns]]
df_scarc_water = df_food.loc[:, ["Scarcity" in i for i in df_food.columns]]

# Correcting the inconsistent naming patterns and renaming the long-winded columns
df_food.rename(columns = {
    'Food product' : 'Food_Product',
    'Packging' : 'Packaging',
    'Total_emissions' : 'Total_Emissions',
    'Animal Feed' : 'Animal_Feed',
    df_eutro.columns[0] : "Eutro_Em_1000kcal",
    df_eutro.columns[1] : "Eutro_Em_1kg",
    df_eutro.columns[2] : "Eutro_Em_100gProtein",
    df_freshwater.columns[0] : "Freshwater_1000kcal",
    df_freshwater.columns[1] : "Freshwater_100gProtein",
    df_freshwater.columns[2] : "Freshwater_1kg",
    df_gas.columns[0] : "GGas_Em_1000kcal",
    df_gas.columns[1] : "GGas_Em_100gProtein",
    df_land.columns[0] : "Land_Use_Change",
    df_land.columns[1] : "Land_Use_1000kcal",
    df_land.columns[2] : "Land_Use_1kg",
    df_land.columns[3] : "Land_Use_100gProtein",
    df_scarc_water.columns[0] : "ScarcWater_1kg",
    df_scarc_water.columns[1] : "ScarcWater_100gProtein",
    df_scarc_water.columns[2] : "ScarcWater_1000kcal"
}, inplace=True)

print("New column names: \n")
for col in df_food:
    print(col)

Low number of unique values?

On a secondary nature, it would be a fair assumption to expect different food products to have different CO2 emission measurements for certain features/columns like [Retail], [Transport], [Packaging], and [Animal Feed]. However, they have a surprising low amount of unique values, as can be seen in the following cell.

In [ ]:
print("Unique values - standardized agriculture operations: " + "\n\n"
    + str(df_food[["Retail", "Transport", "Packaging", "Animal_Feed"]].nunique()) + "\n\n"
    + "Top 10 Retail Values for Food Products" + "\n"
    + str(df_food[['Food_Product', 'Retail', 'Transport']].sort_values(by='Retail', ascending=False).head(10)))

Out of 42 entries in our dataset, [Retail] has only four unique values,[Transport] has eight, and [Packaging] and [Animal_Feed] have ten.

This can be justified by considering the following:
1. Primary hypothesis: these features deal with industrialized agricultural techniques, therefore, it is expected that many package-transport-retail processes are standardized.
2. Secondary and less likely hypothesis: these entries may have been assigned values as part of a clustering method rather than on an individual basis.
3. Note that ["Animal_Feed"] concerns only food products with animal food products, therefore it applies only to those 10 entries.

<a id="section-four"></a>
## 4.2 Addressing Negative Values

In [ ]:
# First, let us scout for negative values in our dataset and, if detected, determine how to proceed with the analysis.

df_numeric = df_food.select_dtypes('number')
neg_values = (df_numeric<0).sum().sum()

print(f"Number of negative values in the dataset: {neg_values}")

In [ ]:
# It would appear that only only column holds negative values: Land-Use Change

(df_numeric<0).sum().sort_values(ascending=False).head(3)

In [ ]:
# There are four entries of food products in Land-Use Change that hold negative values.

df_food[df_food['Land_Use_Change']<0][['Food_Product', 'Land_Use_Change']]

As an example of a more deep-dive approach to understanding our dataset let's focus on the most prevalent negative value: why does the "Nuts" entry have a -2.1 value?

As is noted in the website OurWorldInData (https://ourworldindata.org/faqs-environmental-impacts-food):
"In the recent past, many nut plantations have been replaced grasslands or abandoned pastures. Since the trees of nut crops sequester carbon dioxide, when they replace some grasslands this can actually result in an emission saving due to positive land use change. This effect, however, will eventually diminish as nut plantations are grown on land that was not previously grasslands."

Negative values will hinder our ability to properly visualize the data.

Therefore we shall proceed with the following choices:
1. Made a note of our discovery and provided scientific evidence regarding its presence.
2. Replace all negative values with zero so we can proceed with the analysis.

In [ ]:
# We can iterate through the columns in our dataframe that hold numeric values in order to detect and replace all negative values with zero

for col in df_food.iloc[:, df_food.columns.get_loc('Land_Use_Change'):df_food.columns.get_loc('ScarcWater_1000kcal')]:
    for ind, entry in enumerate(df_food[col]):
        if entry < 0:
            df_food.at[ind, col] = 0
print(f"Number of negative values in the dataset: {((df_food.iloc[:,1:-1])<0).sum().sum()}")

In [ ]:
# Creating a food category column which we will retain for all following sections of this analysis.

df_food["Category"] = df_food["Food_Product"] # creating a new column with the exact list of [Food_Products]

# Setting various lists for different types of [Food_Products]
Grains = ["Wheat & Rye (Bread)", "Maize (Meal)", "Oatmeal", "Barley (Beer)", "Rice"]
Nuts = ['Nuts', 'Groundnuts']
Vegetables = ["Potatoes", "Cassava", 'Other Pulses',"Peas",'Tomatoes', 'Onions & Leeks','Root Vegetables',"Brassicas",'Other Vegetables']
Fruits = ['Citrus Fruit', 'Bananas','Apples', 'Berries & Grapes', 'Other Fruit']
Sugars = ['Cane Sugar', 'Beet Sugar',]
Oils = ['Soybean Oil', 'Palm Oil', 'Sunflower Oil', 'Rapeseed Oil', 'Olive Oil']
Dairy = ["Soymilk",'Milk', 'Cheese']
Animal_Prod = ['Beef (beef herd)', 'Beef (dairy herd)','Lamb & Mutton', 'Pig Meat', 'Poultry Meat', 'Eggs', 'Fish (farmed)', 'Shrimps (farmed)']
Other = ["Tofu", "Coffee", "Dark Chocolate", "Wine"]

# Replacing all [Food_Products] in the newly developed column with their respective food [Category]
for i in df_food["Category"]:
    if i in Grains:
        df_food["Category"].replace([i], "Grains", inplace=True)
    elif i in Nuts:
        df_food["Category"].replace([i], "Nuts", inplace=True)
    elif i in Vegetables:
        df_food["Category"].replace([i], "Vegetables", inplace=True)
    elif i in Fruits:
        df_food["Category"].replace([i], "Fruits", inplace=True)
    elif i in Sugars:
        df_food["Category"].replace([i], "Sugar", inplace=True)
    elif i in Oils:
        df_food["Category"].replace([i], "Oils", inplace=True)
    elif i in Dairy:
        df_food["Category"].replace([i], "Dairy", inplace=True)
    elif i in Animal_Prod:
        df_food["Category"].replace([i], "Animal_Prod", inplace=True)
    elif i in Other:
        df_food["Category"].replace([i], "Other", inplace=True)

<a id="section-four"></a>
## 4.3.1 Overview of Missing Values

In the previous overview section we noticed that there are quite a few missing values.

To have a better overview we can create an informative table that will show us the following stats:
1. which columns have missing values
2. their data types
3. the prevalence of missing values based on columns
4. the percentage of missing values based on total length of observations


In [ ]:
n_NAvalues = df_food.isna().sum()
perc_NAvalues = round(df_food.isna().sum()/len(df_food)*100,ndigits=1)

table_NA_info = pd.DataFrame({
    "Data Types": df_food.dtypes,
    "Unique Values" : df_food.nunique(),
    "Total NA Values": n_NAvalues,
    "%Perc NA Values": perc_NAvalues})

table_NA_info.sort_values(by = "%Perc NA Values", ascending = False)

In [ ]:
# As a top-level summary we can explicitly show a key few stats about missing values.
# Approximately 33% of all columns in our dataset have missing values.
# At the highest missing value threshold we have 7 columns with 30% and above missing values,
# 3 columns with 20% to 30% missing values, and 10 columns with 10% to 20% missing values.

upper_NA_threshold = len(table_NA_info[table_NA_info["%Perc NA Values"]>30])
middle_NA_threshold = len(table_NA_info[(table_NA_info["%Perc NA Values"]>20) & (table_NA_info["%Perc NA Values"]<30)])
low_NA_threshold = len(table_NA_info[(table_NA_info["%Perc NA Values"]>10) & (table_NA_info["%Perc NA Values"]>20)])
perc_NA = round((sum(df_food.isna().any())/len(df_food))*100,ndigits=2)

print("Percentage of columns with NA values in original dataset: " + "%" + str(perc_NA) + "\n"
      "Number of columns with 30% to 40% missing values: " + str(upper_NA_threshold) + "\n"
      "Number of columns with 20% to 30% missing values: " + str(middle_NA_threshold) + "\n"
      "Number of columns with 10% to 20% missing values: " + str(low_NA_threshold))

<a id="subsection-four-three-two"></a>
### 4.3.2 MNAR Archetype and systematic bias for missing values

How can we interpret the information we already gathered?

We notice that the majority of missing values in the table_NA_info above are primary found on all columns that deal with protein measurements (37%-40% NA values), followed by all variables dealing with kcal measurements (20% to 30% NA values), and finally by kg measurements (11.6% NA values).

As such, this points towards the "Missing Not At Random" (MNAR) archetype, meaning there is a systematic bias in the missing pattern (i.e. it is not completely random) since the missing values are found in specific columns rather than randomly in our dataset.

<a id="section-four"></a>
### 4.3.3 Imputation Methods

We need to choose an imputation method. Among others, our choices include the following methods:
1. Dropping columns with missing values
2. Imputation by average value 
3. Imputation by median
4. Imputation by K nearest neighbors (KNN)
5. Multiple imputation by chained equations (MICE)
6. Hot-deck A: custom-fitted imputation using average value
7. Hot-deck B: custom-fitted imputation using MICE

In the next section we will explode these imputation methods in different temporary datasets prior to making final choice.

Under each scenario we will choose three columns with the highest amount of missing values (as shown in the [table_NA_info] above) as examples to illustrate a few highlights of the results of each method and make a brief, top-level note of the differences of the mean values of each of these column after every imputation method.

#### 1) Dropping columns with missing values

As shown in section 4.3.1, approximately 32% of our columns have missing data. We could go with the easy route and simply drop all columns with NA values but to waste such valuable information in such a small dataset with few entries (we only have 43 entries of food products!) is not expected to be particularly helpful.

In [ ]:
# Dropping NA values would be easy thing to do, however, it might not be the most scientific approach.

diff = round((len(df_food.dropna())/len(df_food))*100, ndigits=2)
             
print("Number of rows in original dataset: " + str(len(df_food)) + "\n"
      "Number of rows after dropping NA values: " + str(len(df_food.dropna())) + "\n"
      "Percentage difference: " + "%" + str(diff))

# Indeed, if we choose to drop NA values we would lose a staggesting 55.8% of the observations (rows) in our original dataset.


Performing a drop_na function will cause numerous problems including, among others:
1. cause a significant loss of statistical power (even in a dataset with <50 entries)
2. lower the representation of various food products across key features
3. affect the correlations between said features
4. lower the dependability of our regression analyses

#### 2) Imputation by average value (mean)

Imputation using mean or median are two univariate approaches that are particularly popular due to their ease of implementation and in our case are definetely one step above the previous method of simply dropping all columns with missing values.

In [ ]:
# Mean Imputation

df_mean = df_food.copy(deep=True)

df_mean = df_mean.fillna(df_mean.mean())
df_mean_na_sum = df_mean.isna().sum().sum()

print(f"Method: Mean Imputation \n" 
      + f"Number of missing values in dataset: {df_mean_na_sum} \n\n"
      + f"Mean value in Eutro_Em_1000kcal: {round(df_mean['Eutro_Em_1000kcal'].mean(),2)} \n"
      + f"Mean value in Freshwater_100gProtein: {round(df_mean['Freshwater_100gProtein'].mean(),2)} \n"
      + f"Mean value in Eutro_Em_100gProtein: {round(df_mean['Eutro_Em_100gProtein'].mean(),2)}")

In [ ]:
df_mean.describe()

#### 3) Imputation by median value

In [ ]:
# Median Imputation: the process is quite similar to imputation by mean

df_median = df_food.copy(deep=True)

df_median = df_median.fillna(df_median.median())
df_median_na_sum = df_median.isna().sum().sum()

print(f"Method: Median Imputation \n" 
      + f"Number of missing values in dataset: {df_median_na_sum} \n\n"
      
      + f"Median value in Eutro_Em_1000kcal: {(round(df_median['Eutro_Em_1000kcal'].median(),ndigits=1))} \n"
      + f"Mean value in Eutro_Em_1000kcal: {round(df_median['Eutro_Em_1000kcal'].mean(),2)} \n\n"
      
      + f"Median value in Freshwater_100gProtein: {round(df_median['Freshwater_100gProtein'].median(),2)} \n"
      + f"Mean value in Freshwater_100gProtein: {round(df_median['Freshwater_100gProtein'].mean(),2)} \n\n"
      
      + f"Median value in Eutro_Em_100gProtein: {round(df_median['Eutro_Em_100gProtein'].median(),2)}\n"
      + f"Mean value in Eutro_Em_100gProtein: {round(df_median['Eutro_Em_100gProtein'].mean(),2)}")

In [ ]:
df_median.describe()

#### 4) KNN Imputation

Clustering-based imputation is a very useful tool. This is particularly true in datasets like this where the entries of certain variables are naturally grouped together which, in turn, makes predicting of missing values a lot easier.

In [ ]:
df_numeric = df_food.iloc[:,1:-1] # include only numeric columns 

from sklearn.impute import KNNImputer
knn_imputer = KNNImputer(missing_values = np.nan, n_neighbors = 5, weights = "distance", metric = "nan_euclidean")
# the reason we opt for weighted distance is to alleviate at least some of the issues from outliers (e.g. from Food_Product emission values like Beef)
# also, opting for hyperparameter n = 5, hypothesizing that there will be approximately five different overaching food categories for all food products (see the Hot Deck custom approach section for more info)
df_knn = pd.DataFrame(knn_imputer.fit_transform(df_numeric), columns = df_numeric.columns)

df_knn.insert(0, "Food_Product", df_food["Food_Product"]) # re-attaching the excluded [Food_Product] column
df_knn_na_sum = df_knn.isna().sum().sum()

print(f"Method: KNN Imputation \n" 
      + f"Number of missing values in dataset: {df_knn_na_sum} \n\n"
      + f"Mean value in Eutro_Em_1000kcal: {round(df_knn['Eutro_Em_1000kcal'].mean(),2)} \n"
      + f"Mean value in Freshwater_100gProtein: {round(df_knn['Freshwater_100gProtein'].mean(),2)} \n"
      + f"Mean value in Eutro_Em_100gProtein: {round(df_knn['Eutro_Em_100gProtein'].mean(),2)}")

Note that while the KNN imputation enjoys its fair share of popularity, this does not mean it is the optimal solution.

KNN imputation still struggles with certain issues (like outlier values which is a big problem in this dataset) and may require some fine tuning to the "k" hyperparameter. Still, it can be an invaluable ally when dealing with missing values!

#### 5) MICE Imputation

For this section we will make use of the IterativeImputer library which features multivariate imputation.
This is because it utilizes multiple features  as opposed to the univariate imputation from SimpleImputer which deals with missing values using only data from one feature.

Multiple Imputation by Chained Equations (MICE) is currently considered one of the most potent tools for dealing with missing data.

The process involves multiple iteration circles. During each iteration selected missing data can be imputed through multiple predictive models. These range from standard mean imputation to linear regression and random forests.  Note that by default MICE makes use of BayesianRidge model but there exist specific options to instruct the algorithm which predictive method to use.

Each missing value is treated as a depended variable and the remainder data is used to predict it. After each iteration circle imputed values from specific variables that resulted from that iteration's predictive model are used to impute missing data in other variables. The process is completed when all prediction modelling reaches its course by achieving an optimal result of convergence between existed data and missing data.

In [ ]:
# MICE imputation

# Retrieving numeric columns
df_numeric = df_food.select_dtypes('number') # a more explicit option to select numeric columns
df_mice = df_numeric.copy(deep=True)

# Importing necessary libraries
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge

# We will choose the default model used by the MICE algorithm: the Bayesian Ridge model.
mice_imputer = IterativeImputer(
    missing_values = np.nan,
    estimator = BayesianRidge(), 
    initial_strategy = 'mean',   
    imputation_order = 'ascending',
    verbose = 1,
    max_iter = 9)

df_mice = pd.DataFrame(mice_imputer.fit_transform(df_mice), columns = df_mice.columns)
df_mice.insert(0,"Food_Product", df_food["Food_Product"])
df_mice.insert(1,"Category", df_food["Category"])

df_mice_na_sum = df_mice.isna().sum().sum()

print(f"\nMethod: KNN Imputation \n" 
      + f"Number of missing values in dataset: {df_mice_na_sum} \n\n"
      + f"Mean value in Eutro_Em_1000kcal: {round(df_mice['Eutro_Em_1000kcal'].mean(),2)} \n"
      + f"Mean value in Freshwater_100gProtein: {round(df_mice['Freshwater_100gProtein'].mean(),2)} \n"
      + f"Mean value in Eutro_Em_100gProtein: {round(df_mice['Eutro_Em_100gProtein'].mean(),2)}")

#### 6) Hot-Deck A: Custom-fitted imputation by average value

No imputation walkthrough would be complete without a custom-made imputation method created specifically that our unique dataset allowing us more freedom of engineering, albeit at the cost of a standardized methodology.

For this hot-deck method we can try to blend in a few key features from our previous methods:
1. A useful perspective would be to cluster products based on their respective category where each food product would belong within a specific food category (already performed in the previous section)
2. Once all categories are set, we can utilize them to perform various imputation methods based on each product's respective category. We will use these newly created clusters to make accurate predictions about missing values by using several sub-segments of our dataset to predict similar values.
3. Finally, as an optional method we may blend in our previous imputation methodologies (e.g. like MICE) to further strengthen our algorithm.

The [Category] column was created in the previous section. We can use this new column to further strengthen our imputation methodology and avoid certain imputation pitfalls from previous methods (e.g. standard mean imputation) that might affect our predicted values.

For example, mean imputation will take the mean of all entries of [Food_Products] thus resuling in much higher or lower values for our missing values: the [Food_Product] entry entitled [Beef (beef herd)] scores the highest in almost every feature measuring CO2 emissions across the board while [Bananas] score considerably lower.

In [ ]:
# Looking at an example about product differences. In the two entries "Beef" and "Bananas" we can clearly see their massive difference across all features measuring CO2 emissions

(df_food[df_food["Food_Product"].str.contains("beef") | (df_food["Food_Product"] == "Bananas")])

So here's the key question: why would values originating from red meat and animal products be used to predict values of fruits or vegetables? If we are trying to predict missing values for low-CO2 emission food products from Vegetables and we bundle them with the high-CO2 emission from animal products wouldn't the newly imputed data values of vegetables be "unnaturally" higher?

Let us rectify this issue. We will create several sub-datasets, each containing similar types of food [Categories]. These category-specific sub-datasets will be used to impute values for [Food_Products] contained within them. To put it more simply with an example: missing values in [Fruits] will use data from other [Fruits] or [Vegetables] for the imputation method.

Since we already did the work of clustering each product in each [Category] let us make use of a simple mean imputation based on each slice of the newly formed datasets, and finally merge them all together again.

In [ ]:
# Creating sub-datasets, creating a specialized "imputation pool" for clusters of food categories
df_veg_fruits = df_food[
    (df_food["Category"] == "Vegetables") |
    (df_food["Category"] == "Fruits")]
df_veg_fruits_mean = df_veg_fruits.fillna(df_veg_fruits.mean()) # performing a mean imputation for each missing value in our clusters

# repeate the process for all other categories
df_grains_nuts = df_food[
    (df_food["Category"] == "Nuts") |
    (df_food["Category"] == "Grains")]
df_grains_nuts_mean = df_grains_nuts.fillna(df_grains_nuts.mean())

df_anim_dairy = df_food[
    (df_food["Category"] == "Animal_Prod") |
    (df_food["Category"] == "Dairy")]
df_anim_dairy_mean = df_anim_dairy.fillna(df_anim_dairy.mean())

df_oils_sugar_other = df_food[
    (df_food["Category"] == "Oils") |
    (df_food["Category"] == "Sugar") |
    (df_food["Category"] == "Other")]
df_oils_sugar_other_mean = df_oils_sugar_other.fillna(df_oils_sugar_other.mean())

# concatenating all newly created datasets holding specifically imputed values
df_sum = pd.concat([df_veg_fruits_mean,df_grains_nuts_mean,df_anim_dairy_mean,df_oils_sugar_other_mean]).round(2)
df_sum = df_sum.sort_index()

print(f"Method: Hot-Deck, Custom-Fitted Imputation \n" 
      + f"Number of missing values in dataset: {df_sum.isna().sum().sum()} \n\n"
      + f"Mean value in Eutro_Em_1000kcal: {round(df_sum['Eutro_Em_1000kcal'].mean(),2)} \n"
      + f"Mean value in Freshwater_100gProtein: {round(df_sum['Freshwater_100gProtein'].mean(),2)} \n"
      + f"Mean value in Eutro_Em_100gProtein: {round(df_sum['Eutro_Em_100gProtein'].mean(),2)}")

#### 7) Hot-Deck B: custom-fitted imputation by MICE

In [ ]:
# Alternatively we can go one step further, utilizing both dataset-specific knowledge and methodology by adding MICE imputation method to our product segmentation

# Sklearn Imputer Libraries
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge

# Creating vegetables and fruits sub-dataset
df_veg_fruits = df_food[
    (df_food["Category"] == "Vegetables") |
    (df_food["Category"] == "Fruits")]

# Retrieving the index from original sub-dataset (since the Imputer resets the index)
df_veg_fruits_index = df_veg_fruits.index

# Retrieving only numeric columns
df_veg_fruits_num = df_veg_fruits.select_dtypes('number')

# Imputer transformation
mice_imputer = IterativeImputer(
    missing_values = np.nan,
    estimator = BayesianRidge(),
    verbose = 0)
df_veg_fruits_num_mice = pd.DataFrame(mice_imputer.fit_transform(df_veg_fruits_num), columns = df_veg_fruits_num.columns)

# Re-attaching index and numeric columns (as the first 2 columns)
df_veg_fruits_num_mice.set_index(df_veg_fruits_index, inplace = True)
df_veg_fruits_num_mice.insert(0, "Food_Product", df_veg_fruits["Food_Product"])
df_veg_fruits_num_mice.insert(1, "Category", df_veg_fruits["Category"])

print("Number of missing values in df_veg_fruits: " + str(df_veg_fruits_num_mice.isna().sum().sum()))

The process will repeat for all other sub-datasets which will not be explored since the process is similar to the one in HoT-Deck A approach, albeit with a higher level of complexity. This last approach is the most complicated one and does seem like overkill. Since time and computational requirements are ever-present factors we should opt for one of the previous methods. For this analysis we will make use of the MICE approach, however, the custom-fitted and KNN methods would also be acceptable.

Our imputation section is complete.

We have utilized seven different imputation methods, covering standard easy-to-implement models but also more advanced methodologies.

<a id="section-one"></a>
# 5. Visualization and Analysis

<a id="subsection-five-one"></a>
### 5.1 Setting up

As noted in the previous section, we will opt for the MICE imputation using Bayesian Ridge to deal with missing values which is the most elegant and potent option from aforementioned imputation methods.

In [ ]:
# Choosing the MICE imputation as our default DataFrame

df = df_mice

print(df.columns)

In [ ]:
df.describe()

**Operational Features:** There are some additional inconsistences in our dataset. The column [Total_Emissions] does not equate with the respective measurements in what we will call the "Operational Features": Land_Use_Change, Farm, Transport, Retail, Processing, Packaging and Animal Feed. After all, shouldn't a column entitled "Total Emissions" cover a grand sum of all emissions?

After looking at the numeric values of each column it would appear that Retail emissions might actually be excluded from the [Total_Emissions] column. There is still a 0.3 value difference but at the very least we have noticed and partially solved another issue.

In [ ]:
df_operations = df[['Land_Use_Change', 'Animal_Feed','Farm','Processing','Transport','Packaging','Retail']]

print(f"Sum of [Total_Emissions]: {df['Total_Emissions'].sum().sum()}\n" +
      f"Sum of Operational Features: {df_operations.sum().sum().round()}\n" +
      f"Sum without Retail: {df.iloc[:,2:8].sum().sum().round()}")

In [ ]:
# For starters, let's have a top-level picture:

df_categ_group = df.groupby(["Category"]).sum().sort_values(by='Total_Emissions', ascending = False)
df_categ_group

In [ ]:
# Note that we can also utilize the pivot_table command for a similar result
cols_order = list(df.columns.values)
df_piv = df.round(2).pivot_table(cols_order,index='Category', aggfunc="sum")
df_piv = df_piv.reindex(cols_order,axis=1) # re-indexing the columns in the same order as our dataset
df_piv.drop(df_piv.iloc[:,0:2],axis=1) # deleting the NaN columns

As expected, we can see that Animal Products are at the top of list for [Total_Emissions]

In [ ]:
# A quick overview of the primary "operational" features - we shall look into into those in greater detail.

df_operational = df_food.iloc[:,1:9]
df_oper_col = df_operational.select_dtypes('number').columns

for col in df_oper_col:
    fig, ax = plt.subplots(1, 3, figsize=(10,3))
    sns.histplot(data=df_operational, x=col, ax=ax[0]).set(title="Histogram")
    sns.boxplot(data=df_operational, x=col, ax=ax[1]).set(title="Boxplot")
    sns.violinplot(data=df_operational, x=col, ax=ax[2]).set(title="Violin Plot")

We definetely have massive skewness issues and none even remotely resembles a normal distribution.

Let's do a useful overview visualization showing the [Total_Emissions] by each [Food_Product] filtered by the food [Category].

In [ ]:
sns.set_style("darkgrid")
rel_plot = sns.relplot(data=df_food.sort_values(by='Category'), x='Total_Emissions', y='Food_Product',
            hue='Category', palette="bright",
           height=7, aspect=1)
rel_plot.fig.suptitle("Total Emissions by Food Product, filtered by Food Category")

It would appear that our clustering helped us get some much-needed insight. [Animal_Products] are (understandably) more erratic, while [Fruits], [Vegetables] and indeed most other food [Categories] are relatively stable as far as variability is concerned. The aptly named [Other] category that includes Coffee, Wine, Tofu and Dark Chocolate also holds a higher level of variability due to the fact that most of them are processed rather than raw products and, therefore, are expected to have greater fluctuations in measurements (in addition to being disimilar products). Note that the dataset does not mention Coffee Beans (which are actually derived from the pits of a fruit in its natural form) but rather Coffee which in most cases refers to the processed product.

<a id="subsection-five-two"></a>
### 5.2 Emissions by Product Category

In [ ]:
pie_data = df.groupby(['Category'])["Total_Emissions"].sum().sort_values()
labels = list(df["Category"].unique())
explode_vals= [0.5,0.4,0.3,0.2,0.1,0,0,0,0]
color_vals = ["khaki", "darkgreen", "darkorchid", "olive", "aqua", "grey", "gold", "green", "maroon"]

fig= plt.figure(figsize=(13,8), facecolor = '#eae4f2')
plt.pie(pie_data, labels=labels, autopct='%.1f%%', explode = explode_vals, colors = color_vals)

donut = plt.Circle( (0,0), 0.7, color='white')
p = plt.gcf()
p.gca().add_artist(donut)

plt.title('Percentage of Total Emissions, Segmentation by Food Category')
plt.show()

Animal Products appear to produce more [Total_Emissions] than the next three key sources (Fruits, Oil, and Other - processed products).

Next, since we have 43 different [Food Products] we should look at obtaining a top 15 list.

In [ ]:
df_top15_emm = df.groupby(['Food_Product'])['Total_Emissions'].sum().sort_values(ascending=False).head(15)
diff_top15 = round(((df_top15_emm.sum()/df['Total_Emissions'].sum())*100), 1)

labels_15 = df_top15_emm.index
explode_vals= [0,0,0,0,0,0,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5]

fig = plt.figure(figsize=(15,8),facecolor = '#eae4f2')
plt.pie(df_top15_emm, labels=labels_15, autopct='%.1f%%', explode=explode_vals)

donut = plt.Circle((0,0), 0.7, color='white')
p = plt.gcf()
p.gca().add_artist(donut)

plt.title('Total Emissions from the Top 15 Food Products')

print(f"Grand sum of Total_Emissions: {df['Total_Emissions'].sum()}\n" +
     f"Sum of Top 15 Food_Products: {df_top15_emm.sum().round(1)}\n" +
     f"Total Emission percentage of Top 15 Food Products: {diff_top15}%")

plt.show()

The Top 15 list of [Food_Products] amounts for approximately 85% of the total emissions from all 43 products present in our dataset. Since there are many proponents of the idea that pie charts should contain as little slices as possible we should zoom in closer and look at the Top 10 products just in case we can uncover a more snappy statistic for our hypothetical colleagues, customers and/or food and welfare organizations.

In [ ]:
df_top10_emm = df.groupby(['Food_Product'])['Total_Emissions'].sum().sort_values(ascending=False).head(10)
diff_top10 = round(((df_top10_emm.sum()/df['Total_Emissions'].sum())*100),1)

labels = df_top10_emm.index
explode_vals= [0,0,0,0,0,0,0.1,0.15,0.2,0.25]

fig = plt.figure(figsize=(13,8),facecolor = '#eae4f2')
plt.pie(df_top10_emm, labels=labels, autopct='%.1f%%', explode=explode_vals)

donut = plt.Circle((0,0), 0.7, color='white')
p = plt.gcf()
p.gca().add_artist(donut)

plt.title('Total Emissions from the Top 10 Food Products', fontsize=15)

plt.show()

In [ ]:
print(f"Grand sum of Total_Emissions: {df['Total_Emissions'].sum()}\n" +
     f"Sum of Top 10 Food_Products: {df_top10_emm.sum().round(1)}\n" +
     f"Total Emission percentage of Top 10 Food Products: {diff_top10}%")

This looks better. The Top 10 list of [Food_Products] amounts for approximately 75% or 3/4ths of the total emissions from all 43 products present in our dataset. This is an interesting discovery to help us narrow down the list of products and is a lot more... "catchy" if needed to communicate insights.

<a id="subsection-five-three"></a>
### 5.3 Agriculture Operational Emissions by Value Chain

We should keep looking at the operational features for more information. Let's visualize CO2 emissions in order of value chain from start to finish: Land-Use Change, Farming, Processing, Packaging, Transportation, and finally Retail.

Let us showcase this visualization by keeping it simple and easily readable as dictated by the Zen of Python, and later on we will revisit this visualization and re-forge it using iteration.

In [ ]:
# Visualizing CO2 emissions in order of product value chain without a loop

data_land = df.groupby(['Category'])['Land_Use_Change'].sum().sort_values()
data_farm = df.groupby(['Category'])['Farm'].sum().sort_values()
data_proce = df.groupby(['Category'])['Processing'].sum().sort_values()
data_transp = df.groupby(['Category'])['Transport'].sum().sort_values()
data_pack = df.groupby(['Category'])['Packaging'].sum().sort_values()
data_ret = df.groupby(['Category'])['Retail'].sum().sort_values()

labels = list(df['Category'].unique())
color_vals = ["khaki", "darkgreen", "darkorchid", "olive", "aqua", "grey", "olive", "gold", "brown"]
explode_vals= [0.6,0.5,0.4,0.3,0.2,0.1,0,0,0]

fig, ((ax1,ax2), (ax3,ax4), (ax5, ax6)) = plt.subplots(3,2, figsize=(13,13),facecolor = '#eae4f2')

ax1.pie(data_land, labels=labels, colors=color_vals, explode=explode_vals, autopct='%.1f%%')
ax1.set_title("Land-Use Change Emissions")
ax2.pie(data_farm, labels=labels, colors=color_vals, explode=explode_vals, autopct='%.1f%%')
ax2.set_title("Farm Emissions")
ax3.pie(data_proce, labels=labels, colors=color_vals, explode=explode_vals, autopct='%.1f%%')
ax3.set_title("Processing Emissions")
ax4.pie(data_pack, labels=labels, colors=color_vals, explode=explode_vals, autopct='%.1f%%')
ax4.set_title("Packaging Emissions")
ax5.pie(data_transp, labels=labels, colors=color_vals, explode=explode_vals, autopct='%.1f%%')
ax5.set_title("Transport Emissions")
ax6.pie(data_ret, labels=labels, colors=color_vals, explode=explode_vals, autopct='%.1f%%')
ax6.set_title("Retail Emissions")

plt.suptitle('Greenhouse Emissions by Food Category',fontsize=16)

The greatest disparity appears in [Farm] emissions where [Animal Products] dominate with an astonishing 60.5% of emissions. [Packaging] and [Processing] emissions appear to be relatively more balanced, while [Transport] emissions are by far the most equal across the board (at least compared with other operations). Surprisingly, [Retail] emissions appear to concentrate around three major food categories: Oils, Fruits and Animal Products with a noticeable lack in most other categories.

<a id="subsection-five-four"></a>
### 5.4 Creating and Visualizing a Normalized Variable Index

**Why build a scientific index?** Due to the high number of features in our dataset it would be quite burdersome to visualize each and every one for both [Food_Products] and [Categories]. However, this is not a difficulty that stems from the part of the execution but rather from the point of view of the reader (i.e. our colleagues, science teams, engineering, management, etc.). Note that we have multiple columns measuring emissions based on kilocalories, kilograms and protein per 100 grams.

AS scientists our job is condence information into a meaningful format for our colleagues.

We could easily visualize all 20+ columns but then the readers and colleagues would have to scavenge for an insightful summary among dozens of graphics. 

Instead, let us create something more... digestible (pun intended) about our food product dataset: 
1) create a new normalized feature

2) capable of summarizing information

3) constructed through multiple other features

4) scaled as a min-max index between 0 and 10 with the standard formula

(Note that "index" here refers to a scientific unit of measurement rather than a dataset index)

**Background Research:** After doing some background research I've ascertained that Eutrophication takes its name from the overabundance of nutrients in water and soil usually caused by overuse of man-made chemicals due to farming practices but also from natural processes without human intervention (the usual culprits usually being the overabundance of phosphorus and nitrogen in soil and waters). Eutrophication can affect plants and algae in soil, lakes, surfaces waters, small and large bodies of water and can be attributed to human farming practices in large agricultural facilities.


**Hypothesis 1A:** the amount of [Freshwater] and [Scarce-water] withdrawls is positively related to the amount of: a) [Farm] and [Land-Use Change] emissions (i.e. more crops and animal products require more land which in turn requires more water withdrawal); b) [Greenhouse] emissions (i.e. more products = more emissions; self-explanatory).

**Hypothesis 1B:** Utilizing a measurement of 1000kcal (instead of 1kg or 100gr of protein) would be more effective to communicate insights across science, business operations, government institutions and NGOs since food products will be judged as a cost-benefit analysis between nutrition (and alleviation of world hunger) versus CO2 emissions. Additionally, it is expect that pure caloric measurements will be more effective in communicating useful solutions for world hunger initiatives rather than kilograms or grams of protein.

**Limitations**: Please note that I have created this index simply for the purposes of examining this dataset - it is not a peer-reviewed formula. Specialized environmental scientists will be able to study emissions, chemical runoff due to eutriphication, land-use complications among other subjects in much greater depth than we can in this specific dataset.

However, I believe it is important that we make an effort to simulate that process with the tools we have available. Although we have a sample with few entries it is still considered adequate. Running a Pearson r correlation satisfies (a) the level of measurement (our variables are continuous) and (b) we do have related pairs, however, c) we have a few issues from outliers (mostly from animal products) which we might not wish to drop (since they are an important part of dietary analysis).

In [ ]:
# Looking into the features that we will use for creation of a generalized index measurement formula

df_corr_wel = df[['Eutro_Em_1000kcal', 'Freshwater_1000kcal', 'GGas_Em_1000kcal', 'ScarcWater_1000kcal', 'Land_Use_1000kcal', 'Land_Use_Change', 'Farm']]

sns.heatmap(df_corr_wel.corr())

In [ ]:
# or a lot more readable for looking at specific features:

df_corr_wel.corr().sort_values(by='Eutro_Em_1000kcal', ascending=False)

**Water-Land-Eutrophication (WEL) Index formula:** ((Freshwater Withdrawals per 1000kcals + Scarce Water Withdrawals per 1000kcals) * Eutrophication Emissions per 1000kcals) + Greenhouse Gas Emissions per 1000kcals / Land_Use_Change per 1000kcals + (Farm Emissions + Land-Use Change)/2

**Min-max normalization formula:** (xi-np.min(x)) / (np.max(x)-np.min(x))

In [ ]:
# Creating a new column with the WEL index

df.insert(2, 'WEL_Index', (((df['Freshwater_1000kcal'] + df['ScarcWater_1000kcal']) * df['Eutro_Em_1000kcal']) + df['GGas_Em_1000kcal'] / (df['Land_Use_1000kcal'] + (df['Farm']+df['Land_Use_Change'])/2)))

In [ ]:
# normalization on a 1-10 scale based on min-max values 

df['WEL_Index'] = (df['WEL_Index']-np.min(df['WEL_Index']))/(np.max(df['WEL_Index'])-np.min(df['WEL_Index']))*10

Now let us create a couple of useful tree maps. Usually, they are not used to display extensive statistical insight, but rather to convey information with ease even to non-specialized readers.

In [ ]:
# Creating a general map tree with the WEL Index for food [Category]

df_wel = pd.DataFrame(df.groupby(['Category'])['WEL_Index'].sum()).sort_values(by='WEL_Index')

plt.rcParams['text.color'] = "black"
plt.rcParams['font.size'] = 13
colors = ['palegreen', 'maroon', 'olive', "gold", 'darkviolet', 'khaki', 'deepskyblue', 'forestgreen', 'brown']
fig = plt.figure(figsize=(12,9))
ax = fig.add_subplot()

sqf.plot(sizes = df_wel['WEL_Index'],
         label = df_wel.index,
         alpha=1, pad=False,
         color = colors,
         ax=ax)

plt.axis('off')

plt.text(78, 100,
         'WEL Index, Product Category',
         fontsize = 20, 
         color='grey', 
         horizontalalignment='center',
         verticalalignment='bottom')
        
plt.show()

In [ ]:
# Creating a map tree with percentages for the emissions of the Top 15 [Food_Products]

df_fpr = pd.DataFrame(df.groupby(['Food_Product'])['WEL_Index'].sum()).sort_values(by='WEL_Index', ascending=False).head(15)

# We will be adding percentage values in this tree map in order to increase its appeal to a wider audience
perc_vals = [f"{i/df_fpr['WEL_Index'].sum()*100:.2f}%" for i in df_fpr['WEL_Index']]
perc_vals = perc_vals[0:len(df_fpr)]

labels = [f"{i[0]}\n{i[1]}" for i in zip(df_fpr['WEL_Index'].index,perc_vals)]

plt.rcParams['text.color'] = "black"
plt.rcParams['font.size'] = 10
colors = ['maroon', 'cyan', 'green', 'red', 'coral', 'gold', 'lawngreen', 'gray', 'azure', 'sienna', 'olive', 'orange', 'blue', 'aqua', 'lime']
fig = plt.figure(figsize=(12,9))
ax = fig.add_subplot()

sqf.plot(sizes = df_fpr['WEL_Index'],
         label = labels,
         alpha=1, pad=False,
         color = colors,
        ax=ax)

plt.axis('off')

# We will add two titles with different fontsizes and different positioning in order to convey additional information.
# They need to be aligned precisely on the top right side of our tree map.
plt.text(72, 100,
         'WEL Index,\nTop 15 Food Product Emissions\n',
         fontsize = 22, 
         color ='grey', 
         horizontalalignment='center',
         verticalalignment='bottom')

plt.text(69, 100,
         '\nPercentage derived from grand total of food products',
         fontsize = 15, 
         color ='grey', 
         horizontalalignment='center',
         verticalalignment='bottom')
        
plt.show()

In [ ]:
# Note: as mentioned earlier, Operational Emissions refers to CO2 emissions from the six features dealing with the Food Product lifecycle.
# These are: Land-Use Change, Farming, Processing, Packaging, Transportation, Retail

y1 = df.groupby(['Category'])['Total_Emissions'].sum().sort_values(ascending=False)
x1 = y1.index

y2 = df.groupby(['Food_Product'])['WEL_Index'].sum().sort_values(ascending=False).head(5)
x2 = y2.index

y3 = df.groupby(['Food_Product'])['WEL_Index'].sum().sort_values(ascending=False).tail(5)
x3 = y3.index

fig = plt.figure(figsize=(13,5))

ax1 = fig.add_axes([0.1,0.2,0.95,0.95])
ax2 = fig.add_axes([0.7,0.78,0.6,0.3])
ax3 = fig.add_axes([0.7,0.35,0.6,0.3])

# All food [Categories] are color-coded accordingly (e.g. all animal products are variations of red, vegetables variations of green, etc.)
colors1 = ['darkred', 'black','olive','darkturquoise','khaki','forestgreen','purple','gold','orange']
colors2 = ['darkred', 'darkred','forestgreen', 'darkred','darkred']
colors3 = ['forestgreen','gold','navy','olive','olive']

ax1.bar(x=x1, height=y1, color=colors1)
ax1.spines.right.set_visible(False)
ax1.spines.top.set_visible(False)
ax1.set_title("Total Operational CO2 Emissions \n(Color-Filtered by Product Category)", alpha=1)
ax2.bar(x=x2, height=y2, alpha=0.85, color=colors2)
ax2.set_title("WEL Index CO2 Emissions: Top Five Products", alpha=0.85)
ax3.bar(x=x3, height=y3, alpha=0.85, color=colors3)
ax3.set_title("WEL Index CO2 Emissions: Bottom Five Products", alpha=0.85)
plt.show()

Finally, let us revisit the previous pie chart cluster visualization we generated earlier and re-create a grand overview, only this time we will be utilizing iteration.

In [ ]:
data_emm = df.groupby(['Category'])['Total_Emissions'].sum().sort_values()
data_wel = df.groupby(["Category"])['WEL_Index'].sum().sort_values()

# adding the previously created sub-dfs to a new summary df
df_sum = pd.concat([data_farm, data_proce, data_transp, data_pack, data_transp, data_ret, data_emm, data_wel], axis=1)

df_sum_t = df_sum.T.round(2) # Transpose the grouped dataframe and round to 2 decimal points

In [ ]:
# In the following section the part: ax = axes[row_number // 3, row_number % 3] is a crucial cog in the iteration and quite necessary to the position of each chart.
# If I may assist you, the reader, understand its function better consider the following simple loop:

for i in range(9):
    print(i//3,i%3)

# The output reveals the row,col position of each chart - hopefully this will ease the understanding of the following iteration.

In [ ]:
font_color = '#333c43'
color_vals = ["purple", "gold", "brown", "darkgreen", "khaki", "olive", "aqua", "grey", "maroon"]

fig, axes = plt.subplots(3, 3, figsize=(13, 10), facecolor='#e8f4f0')
fig.delaxes(ax = axes[2,2]) # delete the unneeded blank axes from the 3x3 plt.subplots

# iterating through the dataframe 
for row_number, (idx, row) in enumerate(df_sum_t.iterrows()):
    ax = axes[row_number // 3, row_number % 3]
    ax.pie(row, 
           labels=row.values, 
           startangle=30,
           colors=color_vals, 
           textprops={'color':font_color})
    ax.set_title(idx, fontsize=16, color=font_color)
    
    legend = plt.legend([x for x in row.index], 
                        bbox_to_anchor=(1.8, .87), # modifying position of legend
                        loc='upper left',  
                        ncol=1)
    for text in legend.get_texts():
        plt.setp(text, color=font_color) # changing legend color

fig.subplots_adjust(wspace=.01) # adjusting space between charts

title = fig.suptitle('Total Emissions by Operational Features and WEL Index', y=.95, fontsize=20, color=font_color)

plt.subplots_adjust(top=0.85, bottom=0.15)

Our analysis is complete. We addressed key issues like negative values and missing values, performed various imputation methods, compressed certain choosen features to generate a new overaching index variable, and performed various visualizations aimed at communicating different story-telling insights ranging from food product-specific, to top-tier and low-tier product emissions, easy-to-digest tree maps for quick communication to various stakeholders, as well as an extensive overview of operational emissions across the value chain.

I hope my description was useful to you, the reader, across various sections of the methodology and maybe you picked up a code section or two to mold and adapt to your own analyses!

All the best.